In [1]:
#hide
!pip install voila
!jupyter serverextension enable --sys-prefix voila --debug

Searching ['C:\\Users\\sharo\\source\\repos\\fastai_Apps', 'C:\\Users\\sharo\\.jupyter', 'C:\\Users\\sharo\\anaconda3\\etc\\jupyter', 'C:\\ProgramData\\jupyter'] for config files
Looking for jupyter_config in C:\ProgramData\jupyter
Looking for jupyter_config in C:\Users\sharo\anaconda3\etc\jupyter
Looking for jupyter_config in C:\Users\sharo\.jupyter
Looking for jupyter_config in C:\Users\sharo\source\repos\fastai_Apps
Looking for jupyter serverextension enable_config in C:\ProgramData\jupyter
Looking for jupyter serverextension enable_config in C:\Users\sharo\anaconda3\etc\jupyter
Looking for jupyter serverextension enable_config in C:\Users\sharo\.jupyter
Looking for jupyter serverextension enable_config in C:\Users\sharo\source\repos\fastai_Apps
Paths used for configuration of jupyter_notebook_config: 
    	C:\Users\sharo\anaconda3\etc\jupyter\jupyter_notebook_config.d\jupyterlab.json
    	C:\Users\sharo\anaconda3\etc\jupyter\jupyter_notebook_config.d\voila.json
    	C:\Users\sharo\

In [2]:
#hide
# !pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [3]:
#hide
from fastbook import *
from fastai.vision.widgets import *

In [4]:
import os
key = os.environ.get('AZURE_SEARCH_KEY', '1afa7b4bdcd0457d8f700730b399e197')

In [5]:
# search_images_bing

In [6]:
# results = search_images_bing(key, 'grizzly bear')
# ims = results.attrgot('contentUrl')
# len(ims)

In [7]:
# dest = 'images/grizzly.jpg'
# download_url(ims[0], dest)

In [8]:
bear_types = 'grizzly','black','teddy'
path = Path('bears')

In [9]:
if not path.exists():
    path.mkdir()
    for o in bear_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o} bear')
        download_images(dest, urls=results.attrgot('contentUrl'))

In [10]:
fns = get_image_files(path)
# fns

In [11]:
failed = verify_images(fns)
# failed

In [12]:
failed.map(Path.unlink);

## From Data to DataLoaders

In [13]:
bears = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

### Data Augmentation

## Training Your Model, and Using It to Clean Your Data

In [15]:
bears = bears.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = bears.dataloaders(path)

TypeError: 'NoneType' object is not iterable

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
cleaner = ImageClassifierCleaner(learn)
# cleaner

## Turning Your Model into an Online Application

### Using the Model for Inference

In [ ]:
learn.export()

In [ ]:
path = Path()
path.ls(file_exts='.pkl')

In [ ]:
learn_inf = load_learner(path/'export.pkl')

In [ ]:
# learn_inf.dls.vocab

### Creating a Notebook App from the Model

In [ ]:
btn_upload = widgets.FileUpload()

In [ ]:
out_pl = widgets.Output()

In [ ]:
lbl_pred = widgets.Label()

In [ ]:
btn_run = widgets.Button(description='Classify')

In [ ]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [ ]:
#hide
#Putting back btn_upload to a widget for next cell
btn_upload = widgets.FileUpload()

In [ ]:
VBox([widgets.Label('Select your bear!'), 
      btn_upload, btn_run, out_pl, lbl_pred])

### Turning Your Notebook into a Real App

### Deploying your app

### Further Research

1. Consider how the Drivetrain Approach maps to a project or problem you're interested in.
1. When might it be best to avoid certain types of data augmentation?
1. For a project you're interested in applying deep learning to, consider the thought experiment "What would happen if it went really, really well?"
1. Start a blog, and write your first blog post. For instance, write about what you think deep learning might be useful for in a domain you're interested in.